In [1]:
library('igraph')
packageVersion("igraph")

library("Hmisc")
packageVersion("Hmisc")

library("RColorBrewer")
packageVersion("RColorBrewer")

library("psych")
packageVersion("psych")

library("corrplot")
packageVersion("corrplot")


Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



[1] ‘1.2.4.1’

Loading required package: lattice
Loading required package: survival
Loading required package: Formula
Loading required package: ggplot2

Attaching package: ‘Hmisc’

The following objects are masked from ‘package:base’:

    format.pval, units



[1] ‘4.2.0’

[1] ‘1.1.2’


Attaching package: ‘psych’

The following object is masked from ‘package:Hmisc’:

    describe

The following objects are masked from ‘package:ggplot2’:

    %+%, alpha



[1] ‘1.8.12’

corrplot 0.84 loaded


[1] ‘0.84’

In [2]:
source("./ITS-PhyloseqObjects.R")

Warning message in strsplit(conditionMessage(e), "\n"):
“input string 1 is invalid in this locale”

In [3]:
# 23 samples have counts data
physeq.23s

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 138 taxa and 23 samples ]
sample_data() Sample Data:       [ 23 samples by 11 sample variables ]
tax_table()   Taxonomy Table:    [ 138 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 138 tips and 137 internal nodes ]

In [4]:
# group SVs that are same in Genus level
physeq.23s.genus = tax_glom(physeq.23s, "Genus")
physeq.23s.genus

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 57 taxa and 23 samples ]
sample_data() Sample Data:       [ 23 samples by 11 sample variables ]
tax_table()   Taxonomy Table:    [ 57 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 57 tips and 56 internal nodes ]

In [5]:
# filter out features that are not present in all samples
physeq.23s.genus = prune_taxa(taxa_sums(physeq.23s.genus) > 0, physeq.23s.genus)
physeq.23s.genus

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 36 taxa and 23 samples ]
sample_data() Sample Data:       [ 23 samples by 11 sample variables ]
tax_table()   Taxonomy Table:    [ 36 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 36 tips and 35 internal nodes ]

In [6]:
sample_sums(physeq.23s.genus)

T1R1  T1R4  T1R7  T2R1  T2R4  T2R5  T2R7  T2R9  T3R1  T3R4  T3R5  T3R7  T3R9 
    1     5     5 15284 15245 13983 11441 22513 22161 15343 33787 21855  5877 
 T4R1  T4R4  T4R5  T4R7  T4R9  T5R1  T5R4  T5R5  T5R7  T5R9 
34868 26305 35341 24463  9453 24793 18362 31652 15401  4522

In [7]:
# convert to relative abundance
p.23s.genus.rel = transform_sample_counts(physeq.23s.genus, function(x) 100 * x/sum(x))
sample_sums(p.23s.genus.rel)

T1R1 T1R4 T1R7 T2R1 T2R4 T2R5 T2R7 T2R9 T3R1 T3R4 T3R5 T3R7 T3R9 T4R1 T4R4 T4R5 
 100  100  100  100  100  100  100  100  100  100  100  100  100  100  100  100 
T4R7 T4R9 T5R1 T5R4 T5R5 T5R7 T5R9 
 100  100  100  100  100  100  100

## Relative, MDF

In [8]:
# reduce the name length: p.rel.mdf = p.23s.genus.rel.mdf
p.rel.mdf = subset_samples(p.23s.genus.rel, Material == "MDF")
# filter out features that are not present in all samples
p.rel.mdf = prune_taxa(taxa_sums(p.rel.mdf) > 0, p.rel.mdf)
# filter out fatures that are NA at Genus level
p.rel.mdf = subset_taxa(p.rel.mdf, !Genus == "g__unidentified")
p.rel.mdf

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 16 taxa and 14 samples ]
sample_data() Sample Data:       [ 14 samples by 11 sample variables ]
tax_table()   Taxonomy Table:    [ 16 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 16 tips and 15 internal nodes ]

In [9]:
# log10(x+1) transformation
rel.mdf = otu_table(p.rel.mdf)
rel.mdf = log10(rel.mdf + 1)

In [10]:
# combine feature table with taxonomy
rel.mdf.w.taxa = cbind(rel.mdf, tax_table(p.rel.mdf))

In [11]:
rel.mdf.w.taxa

,T1R1,T1R4,T2R1,T2R4,T2R5,T3R1,T3R4,T3R5,T4R1,T4R4,⋯,T5R1,T5R4,T5R5,Kingdom,Phylum,Class,Order,Family,Genus,Species
SH1726759.08FU_AF444541_refs,0,0,0,0,0,0.0135058699119286,0,0.00637985562721551,0,0,⋯,0,0,0,k__Fungi,p__Basidiomycota,c__Microbotryomycetes,o__Sporidiobolales,f__Sporidiobolaceae,g__Rhodotorula,NA
SH1725251.08FU_KJ706901_reps,0,0,0.0458029521415711,0,0,0.184110426023671,0,0,0.0856347713715121,0,⋯,0.0588822020351915,0,0,k__Fungi,p__Basidiomycota,c__Tremellomycetes,o__Filobasidiales,f__Filobasidiaceae,g__Naganishia,NA
SH1674913.08FU_AJ244232_refs,0,0,0.120770208493147,0,0,0.0516726265398625,0,0,0.0110676195463478,0,⋯,0,0,0,k__Fungi,p__Ascomycota,c__Dothideomycetes,o__Dothideales,f__Aureobasidiaceae,g__Aureobasidium,NA
SH1676198.08FU_KF984792_refs,0,0,0,0,0.0152581730277434,0,0,0.00637985562721551,0,0.0130111054834352,⋯,0,0.0252674851649617,0.00545397701352599,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Trichocomaceae,g__Talaromyces,NA
SH1694434.08FU_EU037063_reps,0,0,0,0,0,0.00390186786956742,0,0,0.00618346458419,0,⋯,0,0,0,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Thermoascaceae,g__Byssochlamys,NA
SH1692811.08FU_FJ430773_reps,2.00432137378264,0,1.80538362500547,1.64504570801197,1.76714392024534,1.71289778653094,1.5862228441217,1.24939220876008,1.48114145722001,1.65856530619474,⋯,1.63816598156585,1.67518074753442,1.15946942544332,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Aspergillaceae,g__Penicillium,NA
SH1695393.08FU_U65307_reps,0,2.00432137378264,1.46166054243026,1.68342277101389,1.42608868012308,1.59823199661568,1.76960573350515,1.86216552553796,1.75123776072646,1.70244134640345,⋯,1.33572041179525,1.57966283112818,1.93148537591966,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Aspergillaceae,g__Aspergillus,NA
SH1685996.08FU_AJ292429_refs,0,0,0,0,0,0,0,0,0.00618346458419,0,⋯,0.00695080626413703,0,0,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Hypocreales,f__Cordycipitaceae,g__Lecanicillium,NA
SH1738712.08FU_MG372138_reps,0,0,0,0.778981346852762,0.581802618821262,0,0.588920427208572,0.14044633624793,0,0.1874703303748,⋯,0,0.122792955802824,0.211001103910464,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Hypocreales,f__Hypocreaceae,g__Trichoderma,NA
SH1744621.08FU_KC181928_reps,0,0,0.573037501809179,0.609579360196959,1.06386908038668,0.283422469714102,0.330532535268407,0.986709580583822,0.0668257775713755,0.287574445768126,⋯,0.122774404634178,0.269623305294267,0.360248231833952,k__Fungi,p__Ascomycota,c__Dothideomycetes,o__Capnodiales,f__Cladosporiaceae,g__Cladosporium,NA


In [12]:
# Pearson correlation
cor.rel.mdf = rcorr(t(rel.mdf), type='pearson')

In [16]:
# extract significant values: abs(r)>0.3 & P < 0.05
indices.r.m = which(abs(cor.rel.mdf$r) > 0.3 & cor.rel.mdf$P < 0.05 & lower.tri(cor.rel.mdf$r),arr.ind=TRUE)
# order index by row, in the same order as graph edges
indices.r.m.by.row = indices.r.m[order(indices.r.m[,1]),]

sig.rel.mdf = cor.rel.mdf$r[indices.r.m.by.row]
length(sig.rel.mdf)

[1] 15

In [17]:
sig.rel.mdf

[1]  0.7301434  0.6277869 -0.5735037  0.8994349  0.7591026  0.5567279
 [7]  0.9433940  0.9124377  0.9670584  0.8352804  0.9754478  0.8498938
[13]  0.8557209  0.5488031  0.7122531

In [18]:
# set all other values to 0 because we only want to show significant edges on a correlation plot
cor.rel.mdf$r[which(abs(cor.rel.mdf$r) <= 0.3 & lower.tri(cor.rel.mdf$r), arr.ind=TRUE)]=0
cor.rel.mdf$r[which(cor.rel.mdf$P >= 0.05 & lower.tri(cor.rel.mdf$r), arr.ind=TRUE)]=0

In [19]:
# show vertex label in Genus level
v.names.r.m = rel.mdf.w.taxa[,20]

In [20]:
# color vertex according to Order level
colourCount = length(unique(rel.mdf.w.taxa[,18]))
getPalette = colorRampPalette(brewer.pal(8, "Set2"))
col.r.m = getPalette(colourCount)
my_color.r.m = col.r.m[as.numeric(as.factor(rel.mdf.w.taxa[,18]))]

In [22]:
# create network graph from correlation matrix
nw.rel.mdf <- graph_from_adjacency_matrix(cor.rel.mdf$r, weighted=T, mode="lower", diag=F)
# only show vertexes that have significant values
V(nw.rel.mdf)$name = v.names.r.m
V(nw.rel.mdf)$size = degree(nw.rel.mdf)
E(nw.rel.mdf)$values = sig.rel.mdf
# red for negative correlation, green for positive correlation
E(nw.rel.mdf)$color = ifelse(E(nw.rel.mdf)$values > 0,'#00FF00','#FF0000')

par(bg="white", mar=c(0,0,0,0))
set.seed(123)
pdf('rel_mdf_correlation.pdf', width=20, height=20)
l.r.m <- layout_with_graphopt(nw.rel.mdf)
plot.igraph(nw.rel.mdf,
            layout=l.r.m, 
            vertex.label.cex=0.9,
            vertex.color=my_color.r.m)
legend(x="topleft", legend=levels(as.factor(rel.mdf.w.taxa[,18])), col = col.r.m, pch=20, cex = 2, text.col="black", horiz = F)
dev.off()

pdf 
  2

## Quantitative, MDF

In [23]:
# using previous objects, and convert relative abundance to quantitative abundance
#sample_data(p.rel.mdf)
count.mdf = as.data.frame(sample_data(p.rel.mdf))$Counts
p.quan.mdf = p.rel.mdf
otu_table(p.quan.mdf) = rel_to_quan(p.quan.mdf, count.mdf)

In [24]:
otu_table(p.quan.mdf)

,T1R1,T1R4,T2R1,T2R4,T2R5,T3R1,T3R4,T3R5,T4R1,T4R4,T4R5,T5R1,T5R4,T5R5
SH1726759.08FU_AF444541_refs,0.000,0.0000,0.000000,0.00000,0.0000000,0.10865936,0.0000000,0.01166622,0.00000000,0.00000000,0.00000000,0.0000000,0.0000000,0.00000000
SH1725251.08FU_KJ706901_reps,0.000,0.0000,2.869668,0.00000,0.0000000,1.81616353,0.0000000,0.00000000,0.37489962,0.00000000,0.00000000,2.8721010,0.0000000,0.00000000
SH1674913.08FU_AJ244232_refs,0.000,0.0000,8.271395,0.00000,0.0000000,0.43463743,0.0000000,0.00000000,0.04439601,0.00000000,0.00000000,0.0000000,0.0000000,0.00000000
SH1676198.08FU_KF984792_refs,0.000,0.0000,0.000000,0.00000,0.8033863,0.00000000,0.0000000,0.01166622,0.00000000,0.05230945,0.00000000,0.0000000,0.1545583,0.04981254
SH1694434.08FU_EU037063_reps,0.000,0.0000,0.000000,0.00000,0.0000000,0.03104553,0.0000000,0.00000000,0.02466445,0.00000000,0.02555106,0.0000000,0.0000000,0.00000000
SH1692811.08FU_FJ430773_reps,2398.786,0.0000,1622.375033,1243.48836,1291.8450976,174.16542575,32.3081536,13.21083059,50.35987152,76.63987835,41.31606350,840.0097608,119.5438405,52.96318826
SH1695393.08FU_U65307_reps,0.000,157.6667,721.130594,1361.03391,576.8313309,132.95248409,49.7346021,56.60684580,95.27876563,84.97015776,248.39036813,408.7159279,95.4327415,332.69798672
SH1685996.08FU_AJ292429_refs,0.000,0.0000,0.000000,0.00000,0.0000000,0.00000000,0.0000000,0.00000000,0.02466445,0.00000000,0.00000000,0.3191223,0.0000000,0.00000000
SH1738712.08FU_MG372138_reps,0.000,0.0000,0.000000,144.38071,63.3068369,0.00000000,2.4774816,0.30098855,0.00000000,0.92849268,8.18485612,0.0000000,0.8430454,2.46572097
SH1744621.08FU_KC181928_reps,0.000,0.0000,70.728867,88.44264,237.8023314,3.16664410,0.9809033,6.85740571,0.28610761,1.61505417,2.10370391,6.4622272,2.2200196,5.09333271


In [25]:
# log10(x+1) transformation
quan.mdf <- otu_table(p.quan.mdf)
quan.mdf <- log10(quan.mdf + 1)

In [26]:
# combine feature table with taxonomy
quan.mdf.w.taxa <- cbind(quan.mdf, tax_table(p.quan.mdf))

In [27]:
quan.mdf.w.taxa

,T1R1,T1R4,T2R1,T2R4,T2R5,T3R1,T3R4,T3R5,T4R1,T4R4,⋯,T5R1,T5R4,T5R5,Kingdom,Phylum,Class,Order,Family,Genus,Species
SH1726759.08FU_AF444541_refs,0,0,0,0,0,0.0447981265766298,0,0.00503725010455044,0,0,⋯,0,0,0,k__Fungi,p__Basidiomycota,c__Microbotryomycetes,o__Sporidiobolales,f__Sporidiobolaceae,g__Rhodotorula,NA
SH1725251.08FU_KJ706901_reps,0,0,0.58767366417309,0,0,0.449657870052198,0,0,0.138270992384007,0,⋯,0.587946676513829,0,0,k__Fungi,p__Basidiomycota,c__Tremellomycetes,o__Filobasidiales,f__Filobasidiaceae,g__Naganishia,NA
SH1674913.08FU_AJ244232_refs,0,0,0.967145080605888,0,0,0.156742156306036,0,0,0.0188652030653307,0,⋯,0,0,0,k__Fungi,p__Ascomycota,c__Dothideomycetes,o__Dothideales,f__Aureobasidiaceae,g__Aureobasidium,NA
SH1676198.08FU_KF984792_refs,0,0,0,0,0.256088755194712,0,0,0.00503725010455044,0,0.0221434691976831,⋯,0,0.062415877872979,0.021111758091803,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Trichocomaceae,g__Talaromyces,NA
SH1694434.08FU_EU037063_reps,0,0,0,0,0,0.0132778439262553,0,0,0.0105816684137352,0,⋯,0,0,0,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Thermoascaceae,g__Byssochlamys,NA
SH1692811.08FU_FJ430773_reps,3.3801724629118,0,3.2104188622927,3.09499083779915,3.11154649295501,2.24344838899722,1.52255055815075,1.15261946217204,1.71062392860812,1.89008484624348,⋯,2.92480103628006,2.0811450240231,1.73209760078623,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Aspergillaceae,g__Penicillium,NA
SH1695393.08FU_U65307_reps,0,2.20048569891321,2.85861574488525,3.13418792104257,2.76180108626012,2.1269507718313,1.70530425863929,1.76047409658965,1.9835305136413,1.93434772371707,⋯,2.61248284766605,1.9842245136975,2.52335358645623,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Aspergillaceae,g__Aspergillus,NA
SH1685996.08FU_AJ292429_refs,0,0,0,0,0,0,0,0,0.0105816684137352,0,⋯,0.120285073064953,0,0,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Hypocreales,f__Cordycipitaceae,g__Lecanicillium,NA
SH1738712.08FU_MG372138_reps,0,0,0,2.16250680040508,1.80825714801791,0,0.541264839200747,0.114273472938411,0,0.285217995199357,⋯,0,0.265536038075318,0.539793593831938,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Hypocreales,f__Hypocreaceae,g__Trichoderma,NA
SH1744621.08FU_KC181928_reps,0,0,1.85569396967192,1.95154459451516,2.37803856245005,0.619786406612976,0.296863285034022,0.895279177870887,0.109277306583635,0.417480689933103,⋯,0.872868470147445,0.507858515989195,0.784854891949308,k__Fungi,p__Ascomycota,c__Dothideomycetes,o__Capnodiales,f__Cladosporiaceae,g__Cladosporium,NA


In [28]:
# Pearson correlation
cor.quan.mdf <- rcorr(t(quan.mdf), type='pearson')

In [32]:
# extract significant values: abs(r)>0.3 & P < 0.05
indices.q.m = which(abs(cor.quan.mdf$r) > 0.3 & cor.quan.mdf$P < 0.05 & lower.tri(cor.quan.mdf$r),arr.ind=TRUE)
# order index by row, in the same order as graph edges
indices.q.m.by.row = indices.q.m[order(indices.q.m[,1]),]

sig.quan.mdf = cor.quan.mdf$r[indices.q.m.by.row]
length(sig.quan.mdf)

[1] 28

In [33]:
sig.quan.mdf

[1] 0.6431248 0.6095417 0.5801023 0.5667737 0.5722502 0.6710000 0.6703754
 [8] 0.5418640 0.6299810 0.9287116 0.7873384 0.7058434 0.8128416 0.8721883
[15] 0.7532615 0.8614649 0.9266213 0.6751979 0.9918724 0.7905655 0.8654415
[22] 0.8086626 0.7623064 0.7707949 0.8486135 0.6742181 0.7102088 0.7238982

In [34]:
# set all other values to 0 because we only want to show significant edges on a correlation plot
cor.quan.mdf$r[which(abs(cor.quan.mdf$r) <= 0.3 & lower.tri(cor.quan.mdf$r), arr.ind=TRUE)]=0
cor.quan.mdf$r[which(cor.quan.mdf$P >= 0.05 & lower.tri(cor.quan.mdf$r), arr.ind=TRUE)]=0

In [35]:
# show vertex label in Genus level
v.names.q.m = quan.mdf.w.taxa[,20]

In [36]:
# color vertex according to Order level
colourCount = length(unique(quan.mdf.w.taxa[,18]))
getPalette = colorRampPalette(brewer.pal(8, "Set2"))
col.q.m = getPalette(colourCount)
my_color.q.m = col.q.m[as.numeric(as.factor(quan.mdf.w.taxa[,18]))]

In [38]:
# create network graph from correlation matrix
nw.quan.mdf <- graph_from_adjacency_matrix(cor.quan.mdf$r, weighted=T, mode="lower", diag=F)
# only show vertexes that have significant values
V(nw.quan.mdf)$name = v.names.q.m
V(nw.quan.mdf)$size = degree(nw.quan.mdf)
E(nw.quan.mdf)$values = sig.quan.mdf
# red for negative correlation, green for positive correlation
E(nw.quan.mdf)$color <- ifelse(E(nw.quan.mdf)$values > 0,'#00FF00','#FF0000')

par(bg="white", mar=c(0,0,0,0))
set.seed(123)
pdf('quan_mdf_correlation.pdf', width=20, height=20)
l.q.m <- layout_with_graphopt(nw.quan.mdf)
plot.igraph(nw.quan.mdf,
            layout=l.q.m, 
            vertex.label.cex=0.9,
            vertex.color=my_color.q.m)
legend(x="topleft", legend=levels(as.factor(quan.mdf.w.taxa[,18])), col = col.q.m, pch=20, cex = 2, text.col="black", horiz = F)
dev.off()

pdf 
  2

## Relative, Gypsum

In [39]:
# reduce the name length: p.rel.mdf = p.23s.genus.rel.mdf
p.rel.gyp = subset_samples(p.23s.genus.rel, Material == "Gypsum")
# filter out features that are not present in all samples
p.rel.gyp = prune_taxa(taxa_sums(p.rel.gyp) > 0, p.rel.gyp)
# filter out fatures that are NA at Genus level
p.rel.gyp = subset_taxa(p.rel.gyp, !Genus == "g__unidentified")
p.rel.gyp

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 22 taxa and 9 samples ]
sample_data() Sample Data:       [ 9 samples by 11 sample variables ]
tax_table()   Taxonomy Table:    [ 22 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 22 tips and 21 internal nodes ]

In [40]:
# log10(x+1) transformation
rel.gyp = otu_table(p.rel.gyp)
rel.gyp = log10(rel.gyp + 1)

In [41]:
# combine feature table with taxonomy
rel.gyp.w.taxa = cbind(rel.gyp, tax_table(p.rel.gyp))

In [42]:
rel.gyp.w.taxa

,T1R7,T2R7,T2R9,T3R7,T3R9,T4R7,T4R9,T5R7,T5R9,Kingdom,Phylum,Class,Order,Family,Genus,Species
SH1726759.08FU_AF444541_refs,2.00432137378264,0,0,0,0,0,0,0,0,k__Fungi,p__Basidiomycota,c__Microbotryomycetes,o__Sporidiobolales,f__Sporidiobolaceae,g__Rhodotorula,NA
SH1715510.08FU_KU204769_reps,0,0.315201857389394,0.865084788656747,0.0416545504129116,0,0,0.196244523157484,0,0,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Trichosphaeriales,f__Trichosphaeriaceae,g__Nigrospora,NA
SH1676198.08FU_KF984792_refs,0,0,0.0188746725056404,0,0.0422182063394944,0,0.23536387746396,0,0.345623840397117,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Trichocomaceae,g__Talaromyces,NA
SH1694434.08FU_EU037063_reps,0,0,0.910285806527871,0,0.263342192142712,0,0.0223843461757409,0,0,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Thermoascaceae,g__Byssochlamys,NA
SH1692811.08FU_FJ430773_reps,0,1.43644301177634,1.68011004616749,0.955256984759955,1.34135658033166,0.928221552025799,0.527580683709867,0.875788191699857,0.744723653265533,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Aspergillaceae,g__Penicillium,NA
SH1695393.08FU_U65307_reps,0,1.83114077670608,0.927491244543949,1.47586421260087,0.934797374663984,1.29648188203823,0.238027778853697,1.13714704931421,0.232399239278689,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Aspergillaceae,g__Aspergillus,NA
SH1679963.08FU_KX664387_reps,0,0,0,0,0.0145335456329176,0,0,0,0,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Hypocreales,f__Ophiocordycipitaceae,g__Tolypocladium,NA
SH1711465.08FU_AY554212_refs,0,0,0.00384112955362787,0,0,0,0,0,0,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Hypocreales,f__Nectriaceae,g__Nalanthamala,NA
SH1685996.08FU_AJ292429_refs,0,0,0.0933731442292972,0.00982386337837918,1.00446256053891,0,1.68487174930527,0,1.52428441955511,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Hypocreales,f__Cordycipitaceae,g__Lecanicillium,NA
SH1729543.08FU_FR682361_reps,0,0,0,0,0,0.0341293614826229,0,0,0,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Hypocreales,f__Hypocreales_fam_Incertae_sedis,g__Acremonium,NA


In [43]:
# Pearson correlation
cor.rel.gyp = rcorr(t(rel.gyp), type='pearson')

In [46]:
# extract significant values: abs(r)>0.3 & P < 0.05
indices.r.g = which(abs(cor.rel.gyp$r) > 0.3 & cor.rel.gyp$P < 0.05 & lower.tri(cor.rel.gyp$r),arr.ind=TRUE)
# order index by row, in the same order as graph edges
indices.r.g.by.row = indices.r.g[order(indices.r.g[,1]),]

sig.rel.gyp = cor.rel.gyp$r[indices.r.g.by.row]
length(sig.rel.gyp)

[1] 35

In [47]:
sig.rel.gyp

[1]  0.8607613 -0.6985160  0.6870854  0.9205787  0.9588260  0.8983374
 [7]  0.8432560  0.7137005  0.8174946  0.8919467  0.6935280  0.7589861
[13]  0.6925061  0.8860340  0.8230132  0.7590728  0.7195082  0.7290343
[19]  0.7851402  0.8407531  0.7708203  0.7275824  0.7415702  0.6755484
[25]  0.7446111  0.8821930  0.6958116  0.7688196  1.0000000  0.7290343
[31]  0.7475539  0.9158262  0.7703338  0.7475539  0.6917741

In [48]:
# set all other values to 0 because we only want to show significant edges on a correlation plot
cor.rel.gyp$r[which(abs(cor.rel.gyp$r) <= 0.3 & lower.tri(cor.rel.gyp$r), arr.ind=TRUE)]=0
cor.rel.gyp$r[which(cor.rel.gyp$P >= 0.05 & lower.tri(cor.rel.gyp$r), arr.ind=TRUE)]=0

In [49]:
# show vertex label in Genus level
v.names.r.g = rel.gyp.w.taxa[,15]

In [50]:
# color vertex according to Order level
colourCount = length(unique(rel.gyp.w.taxa[,13]))
getPalette = colorRampPalette(brewer.pal(8, "Set2"))
col.r.g = getPalette(colourCount)
my_color.r.g = col.r.g[as.numeric(as.factor(rel.gyp.w.taxa[,13]))]

In [52]:
# create network graph from correlation matrix
nw.rel.gyp <- graph_from_adjacency_matrix(cor.rel.gyp$r, weighted=T, mode="lower", diag=F)
# only show vertexes that have significant values
V(nw.rel.gyp)$name = v.names.r.g
V(nw.rel.gyp)$size = degree(nw.rel.gyp)
E(nw.rel.gyp)$values = sig.rel.gyp
# red for negative correlation, green for positive correlation
E(nw.rel.gyp)$color <- ifelse(E(nw.rel.gyp)$values > 0,'#00FF00','#FF0000')

par(bg="white", mar=c(0,0,0,0))
set.seed(123)
pdf('rel_gyp_correlation.pdf', width=20, height=20)
l.r.g <- layout_with_graphopt(nw.rel.gyp)
plot.igraph(nw.rel.gyp,
            layout=l.r.g, 
            vertex.label.cex=0.9,
            vertex.color=my_color.r.g)
legend(x="topleft", legend=levels(as.factor(rel.gyp.w.taxa[,13])), col = col.r.g, pch=20, cex = 2, text.col="black", horiz = F)
dev.off()

pdf 
  2

## Quantitative, Gypsum

In [53]:
# using previous objects, and convert relative abundance to quantitative abundance
#sample_data(p.rel.gyp)
count.gyp = as.data.frame(sample_data(p.rel.gyp))$Counts
p.quan.gyp = p.rel.gyp
otu_table(p.quan.gyp) = rel_to_quan(p.quan.gyp, count.gyp)

In [54]:
otu_table(p.quan.gyp)

,T1R7,T2R7,T2R9,T3R7,T3R9,T4R7,T4R9,T5R7,T5R9
SH1726759.08FU_AF444541_refs,1018.769,0.00000,0.0000000,0.000,0.00000,0.000,0.00000,0.0000,0.000
SH1715510.08FU_KU204769_reps,0.000,269.61350,314.3633678,1052.313,0.00000,0.000,702.37312,0.0000,0.000
SH1676198.08FU_KF984792_refs,0.000,0.00000,2.2060587,0.000,41.10314,0.000,884.46985,0.0000,20905.413
SH1694434.08FU_EU037063_reps,0.000,0.00000,354.2930307,0.000,335.67568,0.000,65.03455,0.0000,0.000
SH1692811.08FU_FJ430773_reps,0.000,6654.14946,2328.0537689,83850.183,8432.99518,231093.209,2913.54774,223494.7772,78300.275
SH1695393.08FU_U65307_reps,0.000,16886.20260,370.6178652,302252.883,3062.18428,580828.584,897.47676,436293.8921,12163.149
SH1679963.08FU_KX664387_reps,0.000,0.00000,0.0000000,0.000,13.70105,0.000,0.00000,0.0000,0.000
SH1711465.08FU_AY554212_refs,0.000,0.00000,0.4412117,0.000,0.00000,0.000,0.00000,0.0000,0.000
SH1685996.08FU_AJ292429_refs,0.000,0.00000,11.9127171,239.162,3665.03040,0.000,58283.96170,0.0000,557604.384
SH1729543.08FU_FR682361_reps,0.000,0.00000,0.0000000,0.000,0.00000,2526.990,0.00000,0.0000,0.000


In [55]:
# log10(x+1) transformation
quan.gyp <- otu_table(p.quan.gyp)
quan.gyp <- log10(quan.gyp + 1)

In [56]:
# combine feature table with taxonomy
quan.gyp.w.taxa <- cbind(quan.gyp, tax_table(p.quan.gyp))

In [57]:
quan.gyp.w.taxa

,T1R7,T2R7,T2R9,T3R7,T3R9,T4R7,T4R9,T5R7,T5R9,Kingdom,Phylum,Class,Order,Family,Genus,Species
SH1726759.08FU_AF444541_refs,3.00850190364752,0,0,0,0,0,0,0,0,k__Fungi,p__Basidiomycota,c__Microbotryomycetes,o__Sporidiobolales,f__Sporidiobolaceae,g__Rhodotorula,NA
SH1715510.08FU_KU204769_reps,0,2.43234945079276,2.4988112448825,3.0225572862837,0,0,2.8471857647954,0,0,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Trichosphaeriales,f__Trichosphaeriaceae,g__Nigrospora,NA
SH1676198.08FU_KF984792_refs,0,0,0.505971472555008,0,1.62431453431391,0,2.94717377796243,0,4.32027952878349,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Trichocomaceae,g__Talaromyces,NA
SH1694434.08FU_EU037063_reps,0,0,2.5505866886008,0,2.52721174781211,0,1.81977120792266,0,0,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Thermoascaceae,g__Byssochlamys,NA
SH1692811.08FU_FJ430773_reps,0,3.82315781337832,3.36717951484708,4.92350919209139,3.92603334851034,5.36378906204029,3.46457117337884,5.3492693218231,4.89376883238578,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Aspergillaceae,g__Penicillium,NA
SH1695393.08FU_U65307_reps,0,4.22755771387085,2.57009658418541,5.48037188883803,3.48617312424158,5.76404872872862,2.95350684770576,5.63978012849284,4.08508174813745,k__Fungi,p__Ascomycota,c__Eurotiomycetes,o__Eurotiales,f__Aspergillaceae,g__Aspergillus,NA
SH1679963.08FU_KX664387_reps,0,0,0,0,1.16734830214606,0,0,0,0,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Hypocreales,f__Ophiocordycipitaceae,g__Tolypocladium,NA
SH1711465.08FU_AY554212_refs,0,0,0.158727792497836,0,0,0,0,0,0,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Hypocreales,f__Nectriaceae,g__Nalanthamala,NA
SH1685996.08FU_AJ292429_refs,0,0,1.1110176362233,2.3805042162979,3.56419606188553,0,4.76555651546099,0,5.74632695822671,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Hypocreales,f__Cordycipitaceae,g__Lecanicillium,NA
SH1729543.08FU_FR682361_reps,0,0,0,0,0,3.40277530131772,0,0,0,k__Fungi,p__Ascomycota,c__Sordariomycetes,o__Hypocreales,f__Hypocreales_fam_Incertae_sedis,g__Acremonium,NA


In [58]:
# Pearson correlation
cor.quan.gyp = rcorr(t(quan.gyp), type='pearson')

In [59]:
# extract significant values: abs(r)>0.3 & P < 0.05
indices.q.g = which(abs(cor.quan.gyp$r) > 0.3 & cor.quan.gyp$P < 0.05 & lower.tri(cor.quan.gyp$r),arr.ind=TRUE)
# order index by row, in the same order as graph edges
indices.q.g.by.row = indices.q.g[order(indices.q.g[,1]),]

sig.quan.gyp = cor.quan.gyp$r[indices.q.g.by.row]
length(sig.quan.gyp)

[1] 33

In [60]:
sig.quan.gyp

[1] -0.8828409 -0.7744219  0.9569149  0.9281510 -0.8443686  0.9869757
 [7]  0.9352157 -0.8689614  0.9587893  0.8572780  0.7129239  0.9785452
[13]  0.9811457  0.9143639  0.7892127  0.9265368  0.9050560  0.9531123
[19] -0.6994710  0.7869726  0.8890662  0.7979407  0.7659547  0.7926035
[25]  0.9828422 -0.6905132  1.0000000  0.8797774  0.8762690  0.7011887
[31]  0.9260659  0.8923061  0.6853609

In [61]:
# set all other values to 0 because we only want to show significant edges on a correlation plot
cor.quan.gyp$r[which(abs(cor.quan.gyp$r) <= 0.3 & lower.tri(cor.quan.gyp$r), arr.ind=TRUE)]=0
cor.quan.gyp$r[which(cor.quan.gyp$P >= 0.05 & lower.tri(cor.quan.gyp$r), arr.ind=TRUE)]=0

In [62]:
# show vertex label in Genus level
v.names.q.g = quan.gyp.w.taxa[,15]

In [63]:
# color vertex according to Order level
colourCount = length(unique(quan.gyp.w.taxa[,13]))
getPalette = colorRampPalette(brewer.pal(8, "Set2"))
col.q.g = getPalette(colourCount)
my_color.q.g = col.q.g[as.numeric(as.factor(quan.gyp.w.taxa[,13]))]

In [65]:
# create network graph from correlation matrix
nw.quan.gyp <- graph_from_adjacency_matrix(cor.quan.gyp$r, weighted=T, mode="lower", diag=F)
# only show vertexes that have significant values
V(nw.quan.gyp)$name = v.names.q.g
V(nw.quan.gyp)$size = degree(nw.quan.gyp)
E(nw.quan.gyp)$values = sig.quan.gyp
# red for negative correlation, green for positive correlation
E(nw.quan.gyp)$color <- ifelse(E(nw.quan.gyp)$values > 0,'#00FF00','#FF0000')

par(bg="white", mar=c(0,0,0,0))
set.seed(123)
pdf('quan_gyp_correlation.pdf', width=20, height=20)
l.q.g <- layout_with_graphopt(nw.quan.gyp)
plot.igraph(nw.quan.gyp,
            layout=l.q.g, 
            vertex.label.cex=0.9,
            vertex.color=my_color.q.g)
legend(x="topleft", legend=levels(as.factor(quan.gyp.w.taxa[,13])), col = col.q.g, pch=20, cex = 2, text.col="black", horiz = F)
dev.off()

pdf 
  2

## Correlation Matrix Visualization

### MDF

In [82]:
mdf.rel.quan = lowerUpper(t(cor.rel.mdf$r), cor.quan.mdf$r)

In [83]:
rownames(mdf.rel.quan) = v.names.r.m
colnames(mdf.rel.quan) = v.names.r.m

In [86]:
pdf('mdf_correlation_matrix.pdf', width=30, height=30)
corrplot(mdf.rel.quan, method="circle", tl.col="black", na.label="square", na.label.col="gray", tl.srt=45, tl.cex=2.0)
dev.off()

pdf 
  2

### Gypsum

In [87]:
gyp.rel.quan = lowerUpper(t(cor.rel.gyp$r), cor.quan.gyp$r)

In [88]:
rownames(gyp.rel.quan) = v.names.r.g
colnames(gyp.rel.quan) = v.names.r.g

In [91]:
pdf('gyp_correlation_matrix.pdf', width=30, height=30)
corrplot(gyp.rel.quan, method="circle", tl.col="black", na.label="square", na.label.col="gray", tl.srt=45, tl.cex=2.0)
dev.off()

pdf 
  2